In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

all_data = pd.read_csv('sales所在周次节假日.csv')
all_data.rename(columns={'店铺名称':'store','地理位置指数':'position','年份':'years','周次':'weeks of year','是否包含法定节假日':'Isholiday','季节':'season','大类':'item','销量':'sales','均价':'prices','促销':'discount'},inplace=True)
all_data['weeks'] = all_data['weeks of year']
a = all_data
week_5 = a.loc[lambda a : a['years'] == 2015]
week_6 = a.loc[lambda a : a['years'] == 2016]
week_6['weeks'] += 53
a = pd.concat([week_5,week_6])

from sklearn import preprocessing

#map店铺名称
name = a['store']
le = preprocessing.LabelEncoder()
le.fit(name)
list(name)
le.transform(name)
tmp = le.transform(name)
a['store'] = tmp

#map合并大类
category = a['item']
le.fit(category)
list(category)
le.transform(category)
tmp = le.transform(category)
a['item'] = tmp

#map季节
season = a['season']
le.fit(season)
list(season)
le.transform(season)
tmp = le.transform(season)
a['season'] = tmp



/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [2]:
df = a
masked_series = (df['years']==2016) & (df['weeks'].isin([53,54,55]))
masked_series2 = (df['years']==2016) & (~(df['weeks'].isin([53,54,55])))
df.loc[(masked_series), 'train_or_test'] = 'test'
df.loc[(masked_series2), 'train_or_test'] = 'no_train'
print('Train shape: {}'.format(df.loc[df['years']==2015,:].shape))
print('Test shape: {}'.format(df.loc[df.train_or_test=='test',:].shape))


Train shape: (8119, 12)
Test shape: (340, 12)


In [3]:
df_n = df.drop(['train_or_test'],axis=1)

#log sales,prices
#df_n['sales'] = np.log1p(df.sales.values)
df_n['prices'] = np.log1p(df.sales.values)
df_n.sample(2)
#df_n = df_n.drop(['prices','discount'],axis=1)

,store,position,years,weeks of year,Isholiday,season,item,sales,prices,discount,weeks
4086,6,1,2016,26,0,2,7,4,1.609438,0.373935,79
2468,4,7,2015,45,0,3,6,20,3.044522,0.429943,45


In [4]:
train = df_n[lambda df_n: df_n['weeks'] < 54]
#train = df_n.sort_values('weeks',ascending='True')
#train = train.drop(['prices','discount'],axis=1)
y_train = train['sales']

In [5]:
#train = df_n[lambda df_n: df_n['years']==2015]
#t1 = df_n[lambda df_n: df_n['weeks']==54]
#t2 = df_n[lambda df_n: df_n['weeks']==55]
#t3 = df_n[lambda df_n: df_n['weeks']==56]
#test = pd.concat([t1,t2,t3])

#y_train = train['sales']
#y_test = test['sales']
#X_train = train.drop('sales', axis=1)
#X_test = test.drop('sales', axis=1)

In [6]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error,r2_score
import xgboost as xgb
import lightgbm as lgb

In [7]:
#Validation function
n_folds = 3

def rmsle_cv(model):
    kf = KFold(n_folds).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [8]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(n_jobs = -1)


In [9]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
#score = rmsle_cv(KRR)
#print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
#score = rmsle_cv(GBoost)
#print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))


Lasso score: 0.0003 (0.0003)

ElasticNet score: 0.0004 (0.0004)

Xgboost score: 7.4984 (9.7318)

LGBM score: 11.9890 (8.0823)



In [32]:
df_test = df_n[lambda df_n: df_n['weeks'] > 53]
df_test_1 = df_test[lambda df_n: df_n['weeks'] <67]
df_test_2_ = df_test[lambda df_n: df_n['weeks'] <79]
df_test_2 = df_test_2_[lambda df_n: df_n['weeks'] >67]
df_test_3_ = df_test[lambda df_n: df_n['weeks'] <92]
df_test_3 = df_test_3_[lambda df_n: df_n['weeks'] >79]
df_test_4 = df_test[lambda df_n: df_n['weeks'] >92]

In [33]:
df_train = df_n[lambda df_n: df_n['weeks'] < 53]
df_train_1 = df_train[lambda df_n: df_n['weeks'] <14]
df_train_2_ = df_train[lambda df_n: df_n['weeks'] <27]
df_train_2 = df_train_2_[lambda df_n: df_n['weeks'] >14]
df_train_3_ = df_train[lambda df_n: df_n['weeks'] <40]
df_train_3 = df_train_3_[lambda df_n: df_n['weeks'] >27]
df_train_4 = df_train[lambda df_n: df_n['weeks'] >40]

In [96]:
df_test_0_x = df_test.drop(['sales'],axis=1)
df_test_0_y = df_test['sales']
df_test_1_x = df_test_1.drop(['sales'],axis=1)
df_test_1_y = df_test_1['sales']
df_test_2_x = df_test_2.drop(['sales'],axis=1)
df_test_2_y = df_test_2['sales']
df_test_3_x = df_test_3.drop(['sales'],axis=1)
df_test_3_y = df_test_3['sales']
df_test_4_x = df_test_4.drop(['sales'],axis=1)
df_test_4_y = df_test_4['sales']

df_train_0_x = df_train.drop(['sales'],axis=1)
df_train_0_y = df_train['sales']
df_train_1_x = df_train_1.drop(['sales'],axis=1)
df_train_1_y = df_train_1['sales']
df_train_2_x = df_train_2.drop(['sales'],axis=1)
df_train_2_y = df_train_2['sales']
df_train_3_x = df_train_3.drop(['sales'],axis=1)
df_train_3_y = df_train_3['sales']
df_train_4_x = df_train_4.drop(['sales'],axis=1)
df_train_4_y = df_train_4['sales']

In [97]:
def rmse_test(model):
    model = model.fit(df_train_0_x,df_train_0_y)
    y_pred = model.predict(df_test_1_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_1_y, y_pred_1) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_1_y, y_pred_1))


In [98]:
#rmse_test(model_lgb)
rmse_test(model_xgb)
#rmse_test(lasso)

The rmse of prediction is: 0.4637341911668404
The R^2 of prediction is: 0.9989833033786261


# 第一次迭代

In [109]:
#去头，尾变
model_xgb.fit(df_train_0_x,df_train_0_y)
y_pred_1 = pd.DataFrame(model_xgb.predict(df_test_1_x))

In [113]:
train_ite_1_x = pd.concat([df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x])
train_ite_1_y = pd.concat([df_train_2_y,df_train_3_y,df_train_4_y,y_pred_1])

In [117]:
def rmse_test(model):
    model = model.fit(train_ite_1_x,train_ite_1_y)
    y_pred_2 = model.predict(df_test_2_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_2_y, y_pred_2) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_2_y, y_pred_2))

#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.8631597239585626
The R^2 of prediction is: 0.991872196124549


In [122]:
#去头，尾不变
train_ite_1_x = pd.concat([df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x])
train_ite_1_y = pd.concat([df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y])

def rmse_test(model):
    model = model.fit(train_ite_1_x,train_ite_1_y)
    y_pred_2 = model.predict(df_test_2_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_2_y, y_pred_2) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_2_y, y_pred_2))

#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.8386044408829846
The R^2 of prediction is: 0.9923280600226732


In [125]:
#不去头，尾变
train_ite_1_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x])
train_ite_1_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,y_pred_1])

def rmse_test(model):
    model = model.fit(train_ite_1_x,train_ite_1_y)
    y_pred_2 = model.predict(df_test_2_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_2_y, y_pred_2) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_2_y, y_pred_2))

#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.3423496734122777
The R^2 of prediction is: 0.9987214117283061


In [127]:
#不去头，尾不变
train_ite_1_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x])
train_ite_1_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y])

def rmse_test(model):
    model = model.fit(train_ite_1_x,train_ite_1_y)
    y_pred_2 = model.predict(df_test_2_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_2_y, y_pred_2) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_2_y, y_pred_2))

#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.30249766040691445
The R^2 of prediction is: 0.999001760109369


# 第二次迭代

In [119]:
#去头，尾变
model_xgb.fit(train_ite_1_x,train_ite_1_y)
y_pred_2 = pd.DataFrame(model_xgb.predict(df_test_2_x))

train_ite_2_x = pd.concat([df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x])
train_ite_2_y = pd.concat([df_train_3_y,df_train_4_y,df_test_1_y,y_pred_2])

def rmse_test(model):
    model = model.fit(train_ite_2_x,train_ite_2_y)
    y_pred_3 = model.predict(df_test_3_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_3_y, y_pred_3) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_3_y, y_pred_3))
    
#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.6711696774492253
The R^2 of prediction is: 0.9953239471710732


In [123]:
#去头，尾不变
train_ite_2_x = pd.concat([df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x])
train_ite_2_y = pd.concat([df_train_3_y,df_train_4_y,df_test_1_y,df_test_2_y])

def rmse_test(model):
    model = model.fit(train_ite_2_x,train_ite_2_y)
    y_pred_3 = model.predict(df_test_3_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_3_y, y_pred_3) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_3_y, y_pred_3))
    
#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.5458709309224247
The R^2 of prediction is: 0.9969068948120442


In [128]:
#不去头，尾变
model_xgb.fit(train_ite_1_x,train_ite_1_y)
y_pred_2 = pd.DataFrame(model_xgb.predict(df_test_2_x))

train_ite_2_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x])
train_ite_2_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y,y_pred_2])

def rmse_test(model):
    model = model.fit(train_ite_2_x,train_ite_2_y)
    y_pred_3 = model.predict(df_test_3_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_3_y, y_pred_3) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_3_y, y_pred_3))
    
#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.39084798962453215
The R^2 of prediction is: 0.9984142653388901


In [129]:
#不去头，尾不变
model_xgb.fit(train_ite_1_x,train_ite_1_y)
y_pred_2 = pd.DataFrame(model_xgb.predict(df_test_2_x))

train_ite_2_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x])
train_ite_2_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y,df_test_2_y])

def rmse_test(model):
    model = model.fit(train_ite_2_x,train_ite_2_y)
    y_pred_3 = model.predict(df_test_3_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_3_y, y_pred_3) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_3_y, y_pred_3))
    
#rmse_test(model_lgb)    
rmse_test(model_xgb)

The rmse of prediction is: 0.3620684668012812
The R^2 of prediction is: 0.9986391941732805


# 第三次迭代

In [120]:
#去头，尾变
model_xgb.fit(train_ite_2_x,train_ite_2_y)
y_pred_3 = pd.DataFrame(model_xgb.predict(df_test_3_x))

train_ite_3_x = pd.concat([df_train_4_x,df_test_1_x,df_test_2_x,df_test_3_x])
train_ite_3_y = pd.concat([df_train_4_y,df_test_1_y,df_test_2_y,y_pred_3])

def rmse_test(model):
    model = model.fit(train_ite_3_x,train_ite_3_y)
    y_pred_4 = model.predict(df_test_4_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_4_y, y_pred_4) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_4_y, y_pred_4))
    
#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 1.098489086285411
The R^2 of prediction is: 0.9945023211697112


In [124]:
#去头，尾不变
train_ite_3_x = pd.concat([df_train_4_x,df_test_1_x,df_test_2_x,df_test_3_x])
train_ite_3_y = pd.concat([df_train_4_y,df_test_1_y,df_test_2_y,df_test_3_y])

def rmse_test(model):
    model = model.fit(train_ite_3_x,train_ite_3_y)
    y_pred_4 = model.predict(df_test_4_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_4_y, y_pred_4) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_4_y, y_pred_4))
    
#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.9996616033824921
The R^2 of prediction is: 0.9954470393705185


In [130]:
#不去头，尾变
model_xgb.fit(train_ite_2_x,train_ite_2_y)
y_pred_3 = pd.DataFrame(model_xgb.predict(df_test_3_x))

train_ite_3_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x,df_test_3_x])
train_ite_3_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y,df_test_2_y,y_pred_3])

def rmse_test(model):
    model = model.fit(train_ite_3_x,train_ite_3_y)
    y_pred_4 = model.predict(df_test_4_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_4_y, y_pred_4) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_4_y, y_pred_4))
    
#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.42811028918357225
The R^2 of prediction is: 0.9991649755275568


In [131]:
#不去头，尾不变
model_xgb.fit(train_ite_2_x,train_ite_2_y)
y_pred_3 = pd.DataFrame(model_xgb.predict(df_test_3_x))

train_ite_3_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x,df_test_3_x])
train_ite_3_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y,df_test_2_y,df_test_3_y])

def rmse_test(model):
    model = model.fit(train_ite_3_x,train_ite_3_y)
    y_pred_4 = model.predict(df_test_4_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_4_y, y_pred_4) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_4_y, y_pred_4))
    
#rmse_test(model_lgb)    
rmse_test(model_xgb)
#rmse_test(model_lasso)

The rmse of prediction is: 0.5683778869936248
The R^2 of prediction is: 0.9985281545199678
